In [ ]:
"""
Author: Yang Hu
1. This file gives a script to constrain cosmological parameters (H0, Omega0, Omegak, MB, etc.)
using strong gravitational lenses (lenses) and type Ia supernovae (SNe) with non-parametric analysis.
2. We use baryon accoustic oscillation data (BAO) to obtain an interpolated curve for Hubble parameter Hz via 
Gaussian Process (GP). GP is carried out using george package.
3. For lenses, we use simulated LSST data. For SNe, we use simulated Roman data. For BAO, we use simulated DESI data.
4. Constraints on parameters are obtained via Markov Chain Monte Carlo (MCMC) using emcee package.
"""

In [ ]:
"""
standard imports for data analysis; astropy.cosmology to compute astrophysical quantities with ease; george to run
GP and emcee to run MCMC
"""

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
from scipy.integrate import simps
from scipy.optimize import minimize
import corner

import george
from george import kernels
import emcee 

from astropy.cosmology import FlatLambdaCDM, FlatwCDM, LambdaCDM, wCDM, w0waCDM

In [ ]:
"""
Load original data. Please check that the relevant data files are named and stored in the way the code below 
specified.
"""

#load&rename lenses data
data1 = pd.read_csv("Data/zlens_zsource_LSSTLike.txt", delimiter=' ', header=None)
zlens = data1[0]
zsource = data1[1]
ddt_err = data1[2]

#load&rename SNe data
data2 = pd.read_csv("Data/lcparam_WFIRST_G10.txt", delimiter=' ', skiprows=1, header=None)
data3 = pd.read_csv("Data/syscov_WFIRST.txt", delimiter=' ', skiprows=0, header=None)
zcmb = data2[1]
mb_err = data2[5]
sys_err = np.array(data3)
D_stat = np.diag(mb_err**2)
C_sys = sys_err.reshape((len(data2), len(data2)))
C = D_stat + C_sys
C_inv = np.linalg.inv(C)

#load&rename BAO data
data4 = pd.read_csv("Data/DESI_HZ_error.txt", delimiter=' ', skiprows=1, header=None) 
zBAO = data4[0] #x
sigHz = data4[1] #yerr


In [ ]:
"""
Sometimes we want more simulated data for lenses. Here we provide a way to genearate more LSST-like data.
The number of lensing events in the original LSST-like data is 310, an estimate of expected observed number
of events in LSST's 10-year survey baseline.
"""

#decide whether to generate data and uncertainty for lenses and whether to use them
generate_data = False
generate_uncertainty = False
save_data = False
#IMPORTANT!!!
#if use_data switch is on, and number of lenses is other than 310, then we must use alterative options in name, 
#ddt_err below and looping in log likelihood function!!!
use_data = False


#first entry for number is the total number of data points we want, second entry is the number of original data
#IMPORTANT!!!
#if number of lenses is other than 310, then we must use alterative options in name, ddt_err below and looping 
#in log likelihood function!!!
number = 3000-310
#choose a python random seed for random processes involved
seed_no = 22

if generate_uncertainty:
    np.random.seed(seed_no)
    pu = np.random.uniform(0.06, 0.1, size=len(data1[0]))

if generate_data:
    np.random.seed(seed_no)
    rand_number = np.random.randint(0, 309, size=number)

if save_data:
    data_temp = np.array(data1)
    data_temp[:, 2]=pu
    for i in rand_number:
        data_temp = np.append(data_temp, [data_temp[i]], axis=0)
    df = pd.DataFrame(np.concatenate(([data_temp[:, 0]], [data_temp[:, 1]], [data_temp[:, 2]])).T)
    df.to_csv(r'Data/zlens_zsource_%sLSSTLike_%s.csv' % ((number+310), seed_no), index=False)

if use_data:
    data_new = pd.read_csv("Data/zlens_zsource_%sLSSTLike_%s.csv" % ((number+310), seed_no), skiprows=1, header=None)
    zlens = data_new[0]
    zsource = data_new[1]
    #alternative
    #ddt_err = data_new[2]/np.sqrt(10)
    ddt_err = data_new[2]
    #alternative
    #name = number+310
    name = ''    
else:
    zlens = data1[0]
    zsource = data1[1]
    ddt_err = data1[2]
    name = ''    

In [ ]:
"""
Since in the original DESI data file we do not have mock Hz values, we still have to generate Hz values by
assuming a mock universe model. If the simulated BAO data file contains measured Hz values, we do not need to do 
so.
The same situation applies to lenses and SNe.
"""

#compute mock Hz from BAO data
H0_mock, Om0_mock, Ok0_mock, w_mock, MB_mock = 72, 0.3, 0.00, -1, -19.2
cosmo_mock = wCDM(H0=H0_mock, Om0=Om0_mock, Ode0=1.-Om0_mock-Ok0_mock, w0=w_mock)
Hz_mock = cosmo_mock.H(z=zBAO).value #y for GP

#compute mock time-delay distance for lenses
dd_mock = cosmo_mock.angular_diameter_distance(z=zlens).value
ds_mock = cosmo_mock.angular_diameter_distance(z=zsource).value
dds_mock = cosmo_mock.angular_diameter_distance_z1z2(z1=zlens, z2=zsource).value
ddt_mock = (1. + zlens) * dd_mock * ds_mock / dds_mock

#compute mock luminosity distance for SNe
dl_mock = cosmo_mock.luminosity_distance(z=zcmb).value
mb_mock = 5*np.log10(dl_mock)+25+MB_mock

In [ ]:
"""
GP interpolate Hz from BAO data using george package
"""

#function to get GP interpolation
def get_gp_interp(zBAO, Hz_mock, sigHz):
    """
    Compute the GP interpolation with the error covariance matrix
    TO DO: Marginalise over kernel hyperparameters
    zBAO: array of z from BAO
    Hz_mock: array of Hz from BAO
    sigHz: array of uncertainty of Hz
    """
    #the 2 optical numerical values of hyperparameters are obtained via MCMC in a seperate script "HzGP_constraint.ipynb"
    gp = george.GP(10.186*np.var(Hz_mock)*kernels.ExpSquaredKernel(1.939),
               mean=np.mean(Hz_mock), white_noise=None)
    Hz = Hz_mock
    Hz_err = sigHz
    #define the redshift region for predicting the GP fit
    z_step = np.linspace(0., max(zBAO), 4000)
    gp.compute(zBAO, Hz_err)
    Hz_gp = gp.predict(Hz, z_step, return_cov=False)
    return z_step, Hz_gp

z_step, Hz_gp = get_gp_interp(zBAO, Hz_mock, sigHz)
#normalise the E(z)
Ez_gp = Hz_gp/Hz_gp[0]
#this is the normalised Hz interpolation curve

In [ ]:
"""
We want to get cosmological distances in likelihood function in a non-parametric way.  
"""

c = 299792.458 #in km/s

#function to get time-delay distance in a non-parametric way
def get_model_indep_dist_ddt(zl, zs, z_step, Ez_gp, h0, ok):
    """
    Doesn't account for errors in the GP fit as of now
    zl: float of z at lenses
    zs: float of z at source to integrate till
    z_step: float, the baseline step for integrating
    Ez_gp: array of normalised Hubble parameter from GP interpolation
    h0: float, a value of Hubble's constant
    ok: float, a value of curvature density parameter
    """
    cond_l = z_step <= zl
    cond_s = z_step <= zs
    Ez_gp_l = Ez_gp[cond_l]
    Ez_gp_s = Ez_gp[cond_s]
    dh = c/h0
    if ok < 0.:
        int_comov_l = simps(1/Ez_gp_l, z_step[cond_l])
        dm_l = dh/np.sqrt(abs(ok))*np.sin(np.sqrt(abs(ok))*int_comov_l)
        da_l = dm_l/(1+zl)
        int_comov_s = simps(1/Ez_gp_s, z_step[cond_s])
        dm_s = dh/np.sqrt(abs(ok))*np.sin(np.sqrt(abs(ok))*int_comov_s)
        da_s = dm_s/(1+zs)
        da_ls = (1/(1+zs)*
        (dm_s*np.sqrt(1+ok*(dm_l/dh)**2)-dm_l*np.sqrt(1+ok*(dm_s/dh)**2))
                )
    elif ok == 0.:
        int_comov_l = simps(1/Ez_gp_l, z_step[cond_l])
        dm_l = dh*int_comov_l
        da_l = dm_l/(1+zl)
        int_comov_s = simps(1/Ez_gp_s, z_step[cond_s])
        dm_s = dh*int_comov_s
        da_s = dm_s/(1+zs)
        da_ls = 1/(1+zs)*(dm_s-dm_l)
    elif ok > 0.:
        int_comov_l = simps(1/Ez_gp_l, z_step[cond_l])
        dm_l = dh/np.sqrt(abs(ok))*np.sinh(np.sqrt(abs(ok))*int_comov_l)
        da_l = dm_l/(1+zl)
        int_comov_s = simps(1/Ez_gp_s, z_step[cond_s])
        dm_s = dh/np.sqrt(abs(ok))*np.sinh(np.sqrt(abs(ok))*int_comov_s)
        da_s = dm_s/(1+zs)
        da_ls = (1/(1+zs)*
        (dm_s*np.sqrt(1+ok*(dm_l/dh)**2)-dm_l*np.sqrt(1+ok*(dm_s/dh)**2))
                )
    ddt = (1.+zl) * da_l * da_s / da_ls
    return ddt

#function to get luminosity distance in a non-parametric way
def get_model_indep_dist_dl(z, z_step, Ez_gp, h0, ok):
    """
    Doesn't account for errors in the GP fit as of now
    z: float of z from SNe 
    z_step: float, the baseline step for integrating
    Ez_gp: array of normalised Hubble parameter from GP interpolation
    h0: float, a value of Hubble's constant
    ok: float, a value of curvature density parameter
    """
    cond = z_step <= z
    Ez_gp = Ez_gp[cond]
    if ok < 0.:
        int_comov = simps(1/Ez_gp, z_step[cond])
        comov_curv = np.sqrt(abs(ok)) * int_comov
        dl_int = c*np.sin(comov_curv) * (1+z) / (np.sqrt(abs(ok)))      
    elif ok > 0.:
        int_comov = simps(1/Ez_gp, z_step[cond])
        comov_curv = np.sqrt(abs(ok)) * int_comov      
        dl_int = c*np.sinh(comov_curv) * (1+z) / (np.sqrt(abs(ok)))  
    elif ok == 0.:
        int_comov = simps(1/Ez_gp, z_step[cond])
        comov_curv = int_comov
        dl_int = c*comov_curv * (1+z)
    dl_int /= h0
    return dl_int

In [ ]:
"""
Constraints are obtained by MCMC and here we define the relevant prior and likelihood functions 
"""

#decide whether to run a maximum likelihood testing before running the full lengthy MCMC analysis
ml_test = True

#use uniform priors for all parameters
def log_prior(theta):
    """
    theta: list of floats, folded cosmological parameters.
    """
    h0, ok, Mb = theta
    if 0. <= h0 <= 150. and -0.5 <= ok <= 0.5 and -38.4 <= Mb <= 0.:
        return 0.0
    else:
        return -np.inf

#use a chi-square likelihood function
def log_likelihood(theta, zlens, zsource, ddt_err, zcmb, C_inv, z_step, Ez_gp): 
    """
    theta: list of floats, folded cosmological parameters.
    zlens: array of z at lens
    zsource: array of z at source
    ddt_err: array of uncertainty of time-delay distance
    zcmb: array of z of cmb, obtained for SNe data
    C_inv: covariance matrix indicating uncertainty of SNe data
    z_step: float, the baseline step for integrating
    Ez_gp: array of normalised Hubble parameter from GP interpolation
    """    
    h0, ok, Mb = theta
    zl_list = list(zlens)
    zs_list = list(zsource)
    ddt_list = [0]*len(zlens)
    for n in range(len(zlens)):
        zl = zl_list[n]
        zs = zs_list[n]
        #alternative looping (slow!!!)
        #zs = zsource[z_list_ddt.index(zl)]
        #ddt[z_list_ddt.index(zl)] = get_model_indep_dist(zl, zs, z_step, Ez_gp, h0, ok)
        ddt_list[n] = get_model_indep_dist_ddt(zl, zs, z_step, Ez_gp, h0, ok)
    ddt = np.array(ddt_list)
    z_list_dl = list(zcmb)
    dl = np.zeros(len(zcmb))
    for z in zcmb:        
        dl[z_list_dl.index(z)] = get_model_indep_dist_dl(z, z_step, Ez_gp, h0, ok)
    #check parameters are physical
    if np.all(ddt >= 0):
        if np.all(dl >=0):
            mb = 5*np.log10(dl)+25+Mb
            del_m = mb_mock - mb
            chi_sq_SNe = np.dot(del_m.T, np.dot(C_inv, del_m))
            chi_sq_lenses = np.sum((ddt-ddt_mock)**2./(ddt*ddt_err)**2.)
            return -0.5*(chi_sq_SNe+chi_sq_lenses)
        else:
            return -np.inf
    else:
        return -np.inf

def log_probability(theta, zlens, zsource, ddt_err, zcmb, C_inv, z_step, Ez_gp):
    """
    theta: list of floats, folded cosmological parameters.
    zlens: array of z at lens
    zsource: array of z at source
    ddt_err: array of uncertainty of time-delay distance
    zcmb: array of z of cmb, obtained for SNe data
    C_inv: covariance matrix indicating uncertainty of SNe data
    z_step: float, the baseline step for integrating
    Ez_gp: array of normalised Hubble parameter from GP interpolation
    """    
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, zlens, zsource, ddt_err, zcmb, C_inv, z_step, Ez_gp)

#run a maximum likelihood test
if ml_test:
    nll = lambda *args: -log_likelihood(*args)
    initial = np.array([75., 0.02, -19.])
    soln = minimize(nll, initial, args=(zlens, zsource, ddt_err, zcmb, C_inv, z_step, Ez_gp))
    H0_ml, Ok_ml, Mb_ml = soln.x
    print("Maximum likelihood estimates:")
    print("H0_ml = {0:.3f}".format(H0_ml))
    print("Ok_ml = {0:.3f}".format(Ok_ml))
    print("MB_ml = {0:.3f}".format(Mb_ml))

In [ ]:
"""
presettings for MCMC
"""

nwalkers = 16
nsamples = 3000

startpos = [70., 0.02, -19.]  # H0, Ok
labels = ["H0", "Ok", "MB"]

In [ ]:
"""
run MCMC using emcee package with the presetting above
"""

#decide whether to run MCMC and save the obtained samples, and whether to load a certain sample file
run_mcmc = True
save_mcmc = True
load_mcmc = True

if run_mcmc:
    pos = startpos + 1e-4 * np.random.randn(nwalkers, len(startpos))
    nwalkers, ndim = pos.shape
    display("Sampling cosmological parameters...")
    sampler = emcee.EnsembleSampler(
        nwalkers, ndim, log_probability, args=[zlens, zsource, ddt_err, zcmb, C_inv, z_step, Ez_gp]
    )
    sampler.run_mcmc(pos, nsamples, progress=True);

if save_mcmc:
    samples = sampler.get_chain()
    flat_samples = sampler.get_chain(discard=500, thin=4, flat=True)
    sample_data = pd.DataFrame(flat_samples)
    
    sample_data.to_csv(
        "GP_samples/%sLSST+Roman_HzGP_%ix%i.csv" 
        % (name, nwalkers, nsamples), 
        index=False, header=labels
    )

if load_mcmc:
    flat_samples = pd.read_csv("GP_samples/%sLSST+Roman_HzGP_%ix%i.csv" 
                               % (name, nwalkers, nsamples), skiprows=1, header=None)

In [ ]:
"""
After loading a sample file, we can plot a corner plot showing how well constraints are placed on each interested
cosmological paramters by the data.
"""

#corner plot
fig = corner.corner(
    flat_samples, labels=labels, quantiles=(0.16, 0.84), show_titles=True, 
    title_fmt='.3f', use_math_text=True, truths=[H0_mock, Ok0_mock, MB_mock]
)
fig.suptitle('Corner plot, %sLSST + Roman, Hz GP fit, %ix%i' % (name, nwalkers, nsamples),
             y=1.02)
fig.savefig("GP_plots/%sLSST+Roman_HzGP_%ix%i_corner.png" 
            % (name, nwalkers, nsamples)
            , bbox_inches='tight'
           )